<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C2-Writing_A_Sniffer/IP_Decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Decoding the IP Layer__*

In its current form, our sniffer receives all of the __IP headers__, along with any higher protocols such as __TCP__, __UDP__, or __ICMP__. The information is packed into binary form and, as shown previously, is quite difficult to understand. Let's work on decoding the IP portion of a packet so that we can pull useful information from it, such as the protocol type (__TCP__, __UDP__, or __ICMP__) and the source and destination IP addresses. This will serve as a foundation for further protocol parsing later on.

If we examine what an actual packet looks like on the network, you should understand how we need to decode the incoming packets. Refer to __Figure 3-1__ for the makeup of an __IP header__.

<div align="center" width="100%">
<img src="https://github.com/damianiRiccardo90/BHP/blob/master/C2-Writing_A_Sniffer/IPv4_Header.png?raw=true" alt="From Client to Server" width="50%">
<p style="text-align:center"><strong><em>Figure 3-1</strong></em></p>
</div>

We will decode the entire IP Header (except the Options field) and extract the protocol type, source, and destination IP address. This means we'll be working directly with the binary, and we'll have to come up with a strategy for separating each part of the IP header using Python.

In Python, there are a couple of ways to get external binary data into a data structure. You can use either the __ctype__ module or the __struct__ module to define the data structure. The __ctype__ module is a foreign function library for Python. It provides a bridge to C-based languages, enabling you to use C-compatible data types and call functions in shared libraries. On the other hand, __struct__ converts between Python values and C structs represented as Python byte object. In other words, the __ctype__ module handles binary data types in addition to providing a lot of other functionalities, while the __struct__ module primarily handles binary data.

You will see both methods used when you explore tool repositories on the web. This esction shows you how to use each one to read an __IPv4 header__ off the network. It's up to you to decide which method you prefer, either will work fine.

# *__The ctypes Module__*

The following code snippet defines a new class, __IP__, that can read a packet and parse the header into its separate fields:

In [ ]:
from ctypes import *
import socket
import struct

class IP(Structure):
    _fields_ = [
        ("version",      c_ubyte,  4),  # 4 bit unsigned char
        ("ihl",          c_ubyte,  4),  # 4 bit unsigned char
        ("tos",          c_ubyte,  8),  # 1 byte char
        ("len",          c_ushort, 16), # 2 byte unsigned short
        ("id",           c_ushort, 16), # 2 byte unsigned short
        ("offset",       c_ushort, 16), # 2 byte unsigned short
        ("ttl",          c_ubyte,  8),  # 1 byte char
        ("protocol_num", c_ubyte,  8),  # 1 byte char
        ("sum",          c_ushort, 16), # 2 byte unsigned short
        ("src",          c_uint32, 32), # 4 byte unsigned int
        ("dst",          c_uint32, 32), # 4 byte unsigned int
    ]

    def __new__(cls, socket_buffer=None):
        return cls.from_buffer_copy(socket_buffer)

    def __init__(self, socket_buffer=None):
        # Human readable IP addresses
        self.src_address = socket.inet_ntoa(struct.pack("<L", self.src))
        self.dst_address = socket.inet_ntoa(struct.pack("<L", self.dst))